## 1.VAEから拡散モデルへ

DDPM : Denoising Diffusion Probabilistic Models を作る。

### VAE の復習
データ生成は、潜在変数を固定の正規分布からサンプリングし、潜在変数から観測変数への変換をNNで行う。また、観測変数から潜在変数への変換もNNを使う。

![](./picture/pict1.png)

### 階層型VAE（潜在変数の階層化）
VAEの潜在変数をT個にしたときの階層型VAEは以下のようなモデル

![](./picture/pict2.png)

ここで示す階層型VAEは、直前の確率変数だけから決定されるモデル。各確率分布にはマルコフ性を仮定し、パラメータの増加を防ぐ。階層化によって複雑な表現を効率的に表せる。

しかし、問題点がある。すべてで２T個の確率分布があり、デコーダが２T個、エンコーダ２T個の計2T個のNNが必要になる。T=1000等になるとこの手法では厳しい。

→T=1000でも１つのNNでモデル化できる**拡散モデル**


### 拡散モデルへ
階層型VAEで２つの点を変更すれば、「拡散モデル」になる。

１，観測変数と潜在変数の次元数は同じ<br>
２，エンコーダは、固定の正規分布のノイズを追加する

![](./picture/pict3.png)

$x_{0}$が観測変数でそれ以外は潜在変数。潜在変数と観測変数の次元数が同じなので、すべて$x$で統一する。<br>
ノイズを追加する過程（拡散過程）は固定のガウスノイズを追加するだけなので、$\phi$のパラメータは不要。

**拡散モデルは、ノイズを除去する処理をNNでモデル化します**

## 2.拡散過程と逆拡散過程

### 拡散過程
拡散過程は、最終時刻Tにおいて「完全なノイズ」になるように前時刻のデータにノイズを加えます。

![](./picture/pict4.png)

$x_{T} \sim N(x_{T}; 0,I)$ に従うように、各時刻で加えるノイズを調節します。時刻ｔで以下のようにノイズを加える。

![](./picture/pict5.png)

$t$は$1\le t \le T$とした整数で、　$\beta _{t}$ はハイパーパラメータとする。（例えば０．０１）<br>

$\beta _{t}$を大きくすると加えるノイズが大きくなる。<br>

Tを大きくして、各時刻ごとに適切な$\beta _{t}$を設定し、（全てでT個）することで、$p(x_{T}) \thickapprox N(x_{T};0,I)$ となる。

逆伝播を行うための変数変換トリックは以下の通り。
![](./picture/pict6.png)


### 逆拡散過程

ノイズを除去する処理の部分。NNで行う。
![](./picture/pict7.png)

全てでT回ノイズ除去を行う必要がある。

個別にNNを使う方法もあるが、拡散モデルでは変数の次元が全て同じなので、各時刻で同じ「１つのNN」で実装できそう。

どの時刻の処理かを区別するために、時刻$t$を入力信号として与える。

![](./picture/pict8.png)

NNは、ノイズを少しだけ除去した画像を$\hat{x}_{t-1}$として出力する。これは推定値であり、時刻ｔのデータから、t-1のデータを推測したという意味。

$p_{\theta}(x_{t-1}|x_{t})$をNNを使ってモデル化する必要がある。ここでは、推定値を平均とする正規分布としてモデル化する。

![](./picture/pict9.png)

構成はできたので次は拡散モデルの学習方法について考える

## 3.ELBOの計算①

やることは、VAEのときと同じで、対数尤度の代わりにELBOを大きくすることを考える。<br>
拡散モデルのELBOを実際に計算するは厳しいので、ELBOを近似して計算することを目指す。

#### ①サンプルサイズT
T個のサンプリングデータを使ってELBOを近似する

まずは、VAEのELBOの復習
![](./picture/pict10.png)

変更点は３つ

![](./picture/pict11.png)

よって、拡散モデルのELBOは

![](./picture/pict13.png)


この式をもとに、ELBOの式展開をしていく。

![](./picture/pict14.png)

最初のイコールはマルコフ性から。パラメータ$\theta$に依存しているところに注目して式変形。

期待値はモンテカルロ法によって近似できるので、$x_{1:T}$をいくつか生成して期待値の中身の平均を求めればよい。例えばモンテカルロ法のサンプルサイズを１とすると、ELBOの近似は次の式

![](./picture/pict16.png)

先程のモデル化の部分を思い出して、式変形していくと、

![](./picture/pict17.png)

よって、目的関数$J(\theta)$は、

$$J(\theta)=-1/2\sum_{t=0}^{T-1}\|x_{t}-\hat x_{t}\|^{2}$$

サンプルサイズTでの目的関数Jの求め方は、

１ 拡散過程によりT個のサンプリングを行い<br>
２ NNをT回適用してノイズ除去を行い<br>
３ 各時刻の二乗誤差$\|x_{t}-\hat x_{t}\|^{2}$を求める

次の方法は、T回サンプリングしなくてもよい方法について見ていく

## 4.ELBOの計算②

#### ②サンプルサイズ２
拡散過程の性質を利用して、元データに対して１度ノイズを追加することで、任意の時刻tの$x_{t}$をサンプリングする。つまり、$q(x_{t}|x_{0})$を解析的に表す。

![](./picture/pict18.png)

先に結論

$q(x_{t}|x_{0})$は次の式で表現できる

![](./picture/pict19.png)

(pf)
まず拡散過程は次の式だった
$$
q(x_t \mid x_{t-1}) = \mathcal{N}(x_t; \sqrt{1 - \beta_t} x_{t-1}, \beta_t \mathbf{I})
$$

ここでは、次の $ \alpha_t $ の記号を使い $ q(x_t \mid x_{t-1}) $ を表します。

$$
\alpha_t = 1 - \beta_t
$$

$$
q(x_t \mid x_{t-1}) = \mathcal{N}(x_t; \sqrt{\alpha_t} x_{t-1}, (1 - \alpha_t) \mathbf{I})
$$


このとき $q(x_t \mid x_{t-1}) $からのサンプリングは、変数変換トリックを使うと次の式で表されます。

$$
\varepsilon_t \sim \mathcal{N}(0, \mathbf{I})
$$

$$
x_t = \sqrt{\alpha_t} x_{t-1} + \sqrt{1 - \alpha_t} \varepsilon_t \tag{8.5}
$$

時刻 $ t $における$\varepsilon_t$は、標準正規分布からサンプリングされます。続いて、式 (8.5) の $ t $に$ t-1$ を代入して次の式を得ます。

$$
\varepsilon_{t-1} \sim \mathcal{N}(0, \mathbf{I})
$$

$$
x_{t-1} = \sqrt{\alpha_{t-1}} x_{t-2} + \sqrt{1 - \alpha_{t-1}} \varepsilon_{t-1} \tag{8.6}
$$

$$
x_t = \sqrt{\alpha_t} x_{t-1} + \sqrt{1 - \alpha_t} \varepsilon_t
$$

これを（８．５）式に代入すると
$$
= \sqrt{\alpha_t} \left( \sqrt{\alpha_{t-1}} x_{t-2} + \sqrt{1 - \alpha_{t-1}} \varepsilon_{t-1} \right) + \sqrt{1 - \alpha_t} \varepsilon_t
$$

$$
= \sqrt{\alpha_t \alpha_{t-1}} x_{t-2} + \sqrt{\alpha_t (1 - \alpha_{t-1})} \varepsilon_{t-1} + \sqrt{1 - \alpha_t} \varepsilon_t \tag{8.7}
$$

ガウスノイズ項は１つにまとめることができて、
$$
\varepsilon \sim \mathcal{N}(0, \mathbf{I})
$$

$$
x_t = \sqrt{\alpha_t \alpha_{t-1}} x_{t-2} + \sqrt{1 - \alpha_t \alpha_{t-1}} \varepsilon \tag{8.8}
$$
この操作を$x_{0}$がでるまで続けると、

$$
x_t = \sqrt{\alpha_t \alpha_{t-1} \cdots \alpha_1} x_0 + \sqrt{1 - \alpha_t \alpha_{t-1} \cdots \alpha_1} \, \varepsilon
$$

$$
= \sqrt{\bar{\alpha}_t} x_0 + \sqrt{1 - \bar{\alpha}_t} \, \varepsilon \tag{8.9}
$$

ここでは次のように $ \alpha_t $ から $\alpha_1$ までの積を $\bar{\alpha}_t $で表すことにします。

$$
\bar{\alpha}_t = \alpha_t \alpha_{t-1} \cdots \alpha_1
$$
よって成り立つ。



これを利用して、先程の目的関数Jを式変形する

![](./picture/pict20.png)

２行目の式変形は期待値の線型性、３行目の式変形は期待値を取る変数のみ分布から取り出せればいいので、必要ない変数を含む分布から取り出さなくても良い。

この式で重要な点は２つの手順でサンプリングできるということ

![](./picture/pict21.png)

目的関数JはT個の和として表現されている。Tが多いと計算量が大きくなるので、「T個の和」を近似したい→**一様分布に関する期待値**



$$
\mathbb{E}_{u(t)}\left[ f(t) \right] = \sum_{t=1}^T u(t) f(t) = \sum_{t=1}^T \frac{1}{T} f(t) = \frac{1}{T} \sum_{t=1}^T f(t)
$$

$$
\sum_{t=1}^T f(t) = T \mathbb{E}_{u(t)}\left[ f(t) \right]
$$

この近似を利用すると、

![](./picture/pict22.png)

二重の期待値は次のようなサンプリングで近似できます。例えばモンテカルロ法のサンプルサイズを１とすると、

$$
t \sim U\{1, T\}
$$

$$
x_{t-1} \sim q(x_{t-1} \mid x_0)
$$

$$
x_t \sim q(x_t \mid x_{t-1})
$$

$$
J(\theta) \approx T \log p_{\theta}(x_{t-1} \mid x_t)
$$

なお、$p_{\theta}(x_{t-1} \mid x_t)$ は次の式で表されます。

$$
\hat{x}_{t-1} = \text{NeuralNet}(x_t, t; \theta)
$$

$$
p_{\theta}(x_{t-1} \mid x_t) = \mathcal{N}(x_{t-1}; \hat{x}_{t-1}, \mathbf{I})
$$

サンプルサイズTのところでも見たように、正規分布の対数尤度は２乗誤差に帰着するので、$J(\theta)$は

$$J(\theta) \approx -T/2\|x_{t}-\hat x_{t}\|^{2}$$

まとめると、サンプルサイズ２の目的関数の計算は

1 一様分布U(1,T)から時刻tサンプリングする<br>
2　$q(x_{t-1}|x_{0})$から$x_{t-1}$をサンプリングし、次に$q(x_{t}|x_{t-1})$から$x_{t}$をサンプリングする<br>
3　NNに$x_{t}$を入力して、$\hat x_{t-1}$ を出力する<br>
4　２乗誤差$\|x_{t-1}-\hat x_{t-1}\|^{2}$を求める<br>



## 5.ELBOの計算③

ここでは、サンプルサイズが１でもELBOを近似する方法について考えます。

keyになるのが、$q(x_{t-1}|x_{t},x_{0})$ という確率分布

![](./picture/pict23.png)


$ q(x_{t-1} \mid x_t, x_0) $ は、数式では次のように表されます。

$$
q(x_{t-1} \mid x_t, x_0) = \mathcal{N}(x_{t-1}; \mu_q(x_t, x_0), \sigma_q^2(t) \mathbf{I}) \tag{8.10}
$$

ここでは以下の記号を使用します。

$$
\alpha_t = 1 - \beta_t
$$

$$
\bar{\alpha}_t = \alpha_t \alpha_{t-1} \cdots \alpha_1
$$

$$
\mu_q(x_t, x_0) = \frac{\sqrt{\alpha_t (1 - \bar{\alpha}_{t-1})} x_t + \sqrt{\bar{\alpha}_{t-1} (1 - \alpha_t)} x_0}{1 - \bar{\alpha}_t}
$$

$$
\sigma_q^2(t) = \frac{(1 - \alpha_t)(1 - \bar{\alpha}_{t-1})}{1 - \bar{\alpha}_t}
$$

![](./picture/pict24.png)
平均ベクトルは上の図のように２点の内分点（ようなもの）として理解できる。

(pf)
$ q(x_{t-1} \mid x_t, x_0) $ の導出を行います。$ q(x_{t-1} \mid x_t, x_0) $ はベイズの定理より次の等式が成り立ちます。

$$
q(x_{t-1} \mid x_t, x_0) = \frac{q(x_t \mid x_{t-1}, x_0) \, q(x_{t-1} \mid x_0)}{q(x_t \mid x_0)}
$$

ここでマルコフ性により $q(x_t \mid x_{t-1}, x_0) = q(x_t \mid x_{t-1}) $ が成り立ちます。

よって、$ q(x_{t-1} \mid x_t, x_0) $ は次の式で表されます。

$$
q(x_{t-1} \mid x_t, x_0) = \frac{q(x_t \mid x_{t-1}) \, q(x_{t-1} \mid x_0)}{q(x_t \mid x_0)}
$$

さて、拡散過程に関する確率分布について、これまで得た式は次のとおりです。

$$
q(x_t \mid x_{t-1}) = \mathcal{N}(x_t; \sqrt{\alpha_t} x_{t-1}, (1 - \alpha_t) \mathbf{I})
$$

$$
q(x_t \mid x_0) = \mathcal{N}(x_t; \sqrt{\bar{\alpha}_t} x_0, (1 - \bar{\alpha}_t) \mathbf{I})
$$

これより $ q(x_{t-1} \mid x_t, x_0) $ は次のように式展開できます。

$$
q(x_{t-1} \mid x_t, x_0) = \frac{\mathcal{N}(x_t; \sqrt{\alpha_t} x_{t-1}, (1 - \alpha_t) \mathbf{I}) \, \mathcal{N}(x_{t-1}; \sqrt{\bar{\alpha}_{t-1}} x_0, (1 - \bar{\alpha}_{t-1}) \mathbf{I})}{\mathcal{N}(x_t; \sqrt{\bar{\alpha}_t} x_0, (1 - \bar{\alpha}_t) \mathbf{I})}
$$

上の式のとおり、3つの正規分布の積と除算によって表されます。
そして、結果的に得られる $ q(x_{t-1} \mid x_t, x_0) $ もまた正規分布になります。

続いて、$q(x_{t-1} \mid x_t, x_0) $ が「どのような正規分布になるか」を計算します。このような場合は、正規分布の指数部分に注目することで、効率的に式展開ができます。たとえば、

$$
\mathcal{N}(x_{t-1}; \sqrt{\alpha_{t-1}} x_0, (1 - \alpha_{t-1}) \mathbf{I})
$$

の指数部分は次の式で表されます。

$$
\mathcal{N}(x_t; \sqrt{\alpha_t} x_{t-1}, (1 - \alpha_t) \mathbf{I})
\propto \exp \left\{ -\frac{1}{2} \left( x_t - \sqrt{\alpha_t} x_{t-1} \right)^{\top} ((1 - \alpha_t) \mathbf{I})^{-1} \left( x_t - \sqrt{\alpha_t} x_{t-1} \right) \right\}
$$

$$
= \exp \left\{ -\frac{1}{2} \frac{\| x_t - \sqrt{\alpha_t} x_{t-1} \|^2}{1 - \alpha_t} \right\}
$$

ここで、$ \propto $ は「比例する」を表す記号です。以上の点を踏まえて$ q(x_{t-1} \mid x_t, x_0) $ の指数部分に注目すると次の式展開ができます。

$$
q(x_{t-1} \mid x_t, x_0) \propto \exp \left( -\frac{1}{2} \left( \frac{\| x_t - \sqrt{\alpha_t} x_{t-1} \|^2}{1 - \alpha_t} + \frac{\| x_{t-1} - \sqrt{\bar{\alpha}_{t-1}} x_0 \|^2}{1 - \bar{\alpha}_{t-1}} - \frac{\| x_t - \sqrt{\bar{\alpha}_t} x_0 \|^2}{1 - \bar{\alpha}_t} \right) \right)
$$

$$
= \exp \left( -\frac{1}{2} \left( \frac{\| x_{t-1} \|^2 - 2 \sqrt{\alpha_t} x_t \cdot x_{t-1} + \alpha_t \| x_t \|^2}{1 - \alpha_t} + \frac{\| x_{t-1} \|^2 - 2 \sqrt{\bar{\alpha}_{t-1}} x_0 \cdot x_{t-1} + \bar{\alpha}_{t-1} \| x_0 \|^2}{1 - \bar{\alpha}_{t-1}} \right) \right)
$$

$$
= \exp \left( -\frac{1}{2} \left( \left( \frac{\alpha_t}{1 - \alpha_t} + \frac{1}{1 - \bar{\alpha}_{t-1}} \right) \| x_{t-1} \|^2 - 2 \left( \frac{\sqrt{\alpha_t}}{1 - \alpha_t} x_t + \frac{\sqrt{\bar{\alpha}_{t-1}}}{1 - \bar{\alpha}_{t-1}} x_0 \right) \cdot x_{t-1} \right) \right) + C(x_t, x_0)
$$

ここで、$q(x_{t-1} \mid x_t, x_0) $ は $ x_t = 1 $ の条件付き確率分布です。ここで $ C(x_t, x_0) $ は $ x_{t-1} $ に関係しない項を表します。続いて、上の式を

$$
\exp \left( -\frac{1}{2} \frac{\| x_{t-1} - \mu_q(x_t, x_0) \|^2}{\sigma_q^2(t)} \right)
$$


$$
\sigma_q^2(t) = \frac{1}{\left( \frac{\alpha_t}{1 - \alpha_t} + \frac{1}{1 - \bar{\alpha}_{t-1}} \right)}
$$

$$
= \frac{(1 - \alpha_t)(1 - \bar{\alpha}_{t-1})}{(1 - \bar{\alpha}_{t-1}) \alpha_t + (1 - \alpha_t)}
$$

$$
= \frac{(1 - \alpha_t)(1 - \bar{\alpha}_{t-1})}{1 - \bar{\alpha}_t}
$$


$$
\mu_q(x_t, x_0) = \frac{\sqrt{\alpha_t} \, x_t + \sqrt{\bar{\alpha}_{t-1}} (1 - \alpha_t) \, x_0}{1 - \bar{\alpha}_t}
$$

$$
= \frac{\sqrt{\alpha_t (1 - \bar{\alpha}_{t-1})} \, x_t + \sqrt{\bar{\alpha}_{t-1} (1 - \alpha_t)} \, x_0}{1 - \bar{\alpha}_t}
$$

これより次の式を得ることができました。

$$
q(x_{t-1} \mid x_t, x_0) = \mathcal{N}\left(x_{t-1}; \frac{\sqrt{\alpha_t (1 - \bar{\alpha}_{t-1})} \, x_t + \sqrt{\bar{\alpha}_{t-1} (1 - \alpha_t)} \, x_0}{1 - \bar{\alpha}_t}, \frac{(1 - \alpha_t)(1 - \bar{\alpha}_{t-1})}{1 - \bar{\alpha}_t} \mathbf{I} \right)
$$


上の式を利用して、サンプルサイズ１のELBOの近似解を求める

目的関数は以下のように式変形できた。

$$
J(\theta) = \sum_{t=1}^T \mathbb{E}_{q(x_{t-1}, x_t \mid x_0)} \left[ \log p_{\theta}(x_{t-1} \mid x_t) \right]
$$

$$
= T \, \mathbb{E}_{u(t)} \left[ J_0 \right]
$$



ここでは、$ J_0 $ = $\mathbb{E}_{q(x_{t-1}, x_t \mid x_0)} \left[ \log p_{\theta}(x_{t-1} \mid x_t) \right]$


目的関数は $ T \, \mathbb{E}_{u(t)}[J_0] $ となります。$\mathbb{E}_{u(t)}[\cdot] $ はモンテカルロ法によって近似できます。
定数項を加えても目的関数を最大化する$\theta$は変わらないので、次の定数項をわざと加えます。

$$
\arg \max_{\theta} J_0 = \arg \max_{\theta} \left( J_0 - \mathbb{E}_{q(x_{t-1}, x_t \mid x_0)} \left[ \log q(x_{t-1} \mid x_t, x_0) \right] \right)
$$

$$
= \arg \max_{\theta} \mathbb{E}_{q(x_{t-1}, x_t \mid x_0)} \left[ \log p_{\theta}(x_{t-1} \mid x_t) - \log q(x_{t-1} \mid x_t, x_0) \right]
$$

$$
= \arg \max_{\theta} \mathbb{E}_{q(x_{t-1}, x_t \mid x_0)} \left[ \log \frac{p_{\theta}(x_{t-1} \mid x_t)}{q(x_{t-1} \mid x_t, x_0)} \right]
$$



$ J_1 = \mathbb{E}_{q(x_{t-1}, x_t \mid x_0)} \left[ \log \frac{p_{\theta}(x_{t-1} \mid x_t)}{q(x_{t-1} \mid x_t, x_0)} \right]$とすると、

$ J_0 $ と $ J_1 $ は最大となる $ \theta $ が同じです。そのため、次の $ J_1 $ を使った式を目的関数にできます。

$$
J(\theta) = T \, \mathbb{E}_{u(t)}[J_1]
$$

続いて、$ J_1 $ を次のように式展開します。

$$
J_1 = \int q(x_{t-1}, x_t \mid x_0) \log \frac{p_{\theta}(x_{t-1} \mid x_t)}{q(x_{t-1} \mid x_t, x_0)} \, dx_{t-1} \, dx_t
$$

$$
= \int q(x_t \mid x_0) \, q(x_{t-1} \mid x_t, x_0) \log \frac{q(x_{t-1} \mid x_t, x_0)}{p_{\theta}(x_{t-1} \mid x_t)} \, dx_{t-1} \, dx_t
$$

$$
= -\mathbb{E}_{q(x_t \mid x_0)} \left[ D_{KL}(q(x_{t-1} \mid x_t, x_0) \parallel p_{\theta}(x_{t-1} \mid x_t)) \right]
$$

ここでは確率の乗法定理を使って展開します。最終的に $ J_1 $ は、$q(x_t \mid x_0) $ に関する KL ダイバージェンスの期待値として表されます。モンテカルロ法を使えば、$ x_t $ のサンプル 1 つだけで近似できます。

また、KL ダイバージェンスは 2 つの確率分布が等しいときに最小値（= 0）を取ります。$ J_1 $ は「マイナスの KL ダイバージェンス」なので、2 つの確率分布が等しい $ p_{\theta}(x_{t-1} \mid x_t) = q(x_{t-1} \mid x_t, x_0) $ のときに最大になります。

つまり、ここでの目標は、$ p_{\theta}(x_{t-1} \mid x_t) $ を $ q(x_{t-1} \mid x_t, x_0) $ に一致させることです。$ q(x_{t-1} \mid x_t, x_0) $ は、式 (8.10) のとおり正規分布として表されることを思い出します。

そのため、$p_{\theta}(x_{t-1} \mid x_t) $ を次のように正規分布としてモデル化します。

$$
\hat{x}_{t-1} = \text{NeuralNet}(x_t, t; \theta)
$$

$$
p_{\theta}(x_{t-1} \mid x_t) = \mathcal{N}(x_{t-1}; \hat{x}_{t-1}, \sigma_q^2(t) \mathbf{I})
$$




$ p_{\theta}(x_{t-1} \mid x_t) $ の平均ベクトルは、NNによって求めます。

そして $ p_{\theta}(x_{t-1} \mid x_t) $ の共分散行列は、$ q(x_{t-1} \mid x_t, x_0) $ と同じ値$ \sigma_q^2(t) \mathbf{I} $ に設定します。

なお、これ以降はNNを $ \mu_{\theta}(x_t, t) $ と簡略化して表すことにします。これにより、次の式で $p_{\theta}(x_{t-1} \mid x_t) $ を表します。

$$
p_{\theta}(x_{t-1} \mid x_t) = \mathcal{N}(x_{t-1}; \mu_{\theta}(x_t, t), \sigma_q^2(t) \mathbf{I})
$$

次に KL ダイバージェンスの計算です。今回は 2 つの正規分布の KL ダイバージェンスであり、これは次の式により解析的に求められます
(7章のコラム、２つの正規分布間のKLダイバージェンスを参照)
$$
D_{KL}(q(x_{t-1} \mid x_t, x_0) \parallel p_{\theta}(x_{t-1} \mid x_t)) = \frac{1}{2 \sigma_q^2(t)} \| \mu_{\theta}(x_t, t) - \mu_q(x_t, x_0) \|^2
$$

以上をまとめると、目的関数は次の式で表されます。

$$
J(\theta) = -T \, \mathbb{E}_{u(t)} \left[ \mathbb{E}_{q(x_{t-1}, x_t \mid x_0)} \left[ \frac{1}{2 \sigma_q^2(t)} \| \mu_{\theta}(x_t, t) - \mu_q(x_t, x_0) \|^2 \right] \right]
$$

損失関数は目的関数にマイナスを付けることで求められます。また、損失関数を定数倍にすることで、オプティマイザの学習率の設定で調整できます。そこで上の式を -$\frac{1}{2}$ 倍した値を損失関数に設定します。

$$
\text{LOSS}(x_0; \theta) = \mathbb{E}_{u(t)} \left[ \mathbb{E}_{q(x_{t-1}, x_t \mid x_0)} \left[ \frac{1}{\sigma_q^2(t)} \| \mu_{\theta}(x_t, t) - \mu_q(x_t, x_0) \|^2 \right] \right]
$$

以上が損失関数の計算方法です。モンテカルロ法（サンプルサイズを1とする）を使うと、損失関数は次のように求められます。

$$
t \sim U\{1, T\}
$$

$$
x_t \sim q(x_t \mid x_0)
$$

$$
\text{LOSS}(x_0; \theta) = \frac{1}{\sigma_q^2(t)} \| \mu_{\theta}(x_t, t) - \mu_q(x_t, x_0) \|^2
$$


このように、損失関数は２乗誤差として表現できる。NNでは $\mu_{\theta}(x_t, t)$ が $\mu_q(x_t, x_0) $　に近づくようにパラメータが学習される

## 6.拡散モデルの学習

![](./picture/pict25.png)

では今回作ったNNでは何を出力するモデルになっているのでしょうか。

![](./picture/pict26.png)

今回のモデルでは、ノイズが少し除去された画像を出力としている。

NNはこれ以外の構成方法も考えられる。

### 元データを復元するNN

時刻ｔでの平均ベクトルは以下のようになっていた。
$$
\mu_q(x_t, x_0) = \frac{\sqrt{\alpha_t (1 - \bar{\alpha}_{t-1})} \, x_t + \sqrt{\bar{\alpha}_{t-1} (1 - \alpha_t)} \, x_0}{1 - \bar{\alpha}_t} \tag{8.11}
$$

そして $ \mu_{\theta}(x_t, t) $ をこの式の形式に合わせるように調整します。それが次の式です。

$$
\mu_{\theta}(x_t, t) = \frac{\sqrt{\alpha_t (1 - \bar{\alpha}_{t-1})} \, x_t + \sqrt{\bar{\alpha}_{t-1} (1 - \alpha_t)} \, \hat{x}_{\theta}(x_t, t)}{1 - \bar{\alpha}_t} \tag{8.12}
$$

先程までは8.11を予測するNNを作成していた。このようにすると、損失関数つまり、KLDは以下のようになる

$$
D_{KL}(q(x_{t-1} \mid x_t, x_0) \parallel p_{\theta}(x_{t-1} \mid x_t)) 
= \frac{1}{2 \sigma_q^2(t)} \| \mu_{\theta}(x_t, t) - \mu_q(x_t, x_0) \|^2
$$

$$
= \frac{1}{2 \sigma_q^2(t)} \left( \frac{\sqrt{\bar{\alpha}_{t-1} (1 - \alpha_t)}}{1 - \bar{\alpha}_t} \right)^2 \| \hat{x}_{\theta}(x_t, t) - x_0 \|^2
$$

KL ダイバージェンスが損失関数となるので、上の式より、損失関数は $\hat{x}_{\theta}(x_t, t) $ と $ x_0 $ の 2 乗誤差として表されます。そのため、$ \hat{x}_{\theta}(x_t, t) $ は $ x_0 $ を「教師データ」として、それと同じ出力になるように学習します。つまり今回のNNで行う処理は、下図のようになる

![](./picture/pict27.png)


### ノイズを予測するNN
$q(x_t \mid x_0)$を思い出すと、

$$
q(x_t \mid x_0) = \mathcal{N}(x_t; \sqrt{\bar{\alpha}_t} x_0, (1 - \bar{\alpha}_t) \mathbf{I})
$$

そして $q(x_t \mid x_0) $ からのサンプルは、次の式で表されることを利用します。

$$
\varepsilon \sim \mathcal{N}(0, \mathbf{I})
$$

$$
x_t = \sqrt{\bar{\alpha}_t} x_0 + \sqrt{1 - \bar{\alpha}_t} \, \varepsilon
$$

これより式変形すると、

$$
x_0 = \frac{x_t - \sqrt{1 - \bar{\alpha}_t} \, \varepsilon}{\sqrt{\bar{\alpha}_t}}
$$

そして、上の式を式 (8.11) に代入して式展開します。

$$
\mu_q(x_t, x_0) = \frac{\sqrt{\alpha_t (1 - \bar{\alpha}_{t-1})} \, x_t + \sqrt{\bar{\alpha}_{t-1} (1 - \alpha_t)} \, x_0}{1 - \bar{\alpha}_t}
$$

$$
= \frac{1}{\sqrt{\alpha_t}} \left( x_t - \frac{1 - \alpha_t}{\sqrt{1 - \bar{\alpha}_t}} \varepsilon \right)
$$

この式に対応させるように$ \mu_{\theta}(x_t, t) $ を書き換えると、次のようになります。

$$
\mu_{\theta}(x_t, t) = \frac{1}{\sqrt{\alpha_t}} \left( x_t - \frac{1 - \alpha_t}{\sqrt{1 - \bar{\alpha}_t}} \, \varepsilon_{\theta}(x_t, t) \right)
$$

ここでは $ \varepsilon_{\theta}(x_t, t) $ がニューラルネットワークの出力を表します。このとき KL ダイバージェンスは次のように計算できます。

$$
D_{KL}(q(x_{t-1} \mid x_t, x_0) \parallel p_{\theta}(x_{t-1} \mid x_t)) = \frac{1}{2 \sigma_q^2(t)} \| \mu_{\theta}(x_t, t) - \mu_q(x_t, x_0) \|^2
$$

$$
= \frac{1}{2 \sigma_q^2(t)} \left( \frac{1 - \alpha_t}{\sqrt{1 - \bar{\alpha}_t}} \right)^2 \| \varepsilon_{\theta}(x_t, t) - \varepsilon \|^2
$$

よって、NNは元データから$x_{t}$を作るときに使うノイズ成分$\epsilon$を予測することになる。

![](./picture/pict28.png)

疑似コードは以下のようになる

![](./picture/pict29.png)


## 7.学習後のデータのサンプリング

$ p_{\theta}(x_{t-1} \mid x_t)  $は次の式で表せれた

$$
p_{\theta}(x_{t-1} \mid x_t) = \mathcal{N}(x_{t-1}; \mu_{\theta}(x_t, t), \sigma_q^2(t) \mathbf{I})
$$

これより $ x_{t-1} \sim p_{\theta}(x_{t-1} \mid x_t) $ によるサンプリングは、変数変換トリックを使うと次の式で表されます。

$$
\varepsilon \sim \mathcal{N}(0, \mathbf{I})
$$

$$
x_{t-1} = \mu_{\theta}(x_t, t) + \sigma_q(t) \varepsilon
$$

ここで $ \mu_{\theta}(x_t, t) $ と$ \sigma_q(t) $ は次の式で表されます。

$$
\mu_{\theta}(x_t, t) = \frac{1}{\sqrt{\alpha_t}} \left( x_t - \frac{1 - \alpha_t}{\sqrt{1 - \bar{\alpha}_t}} \, \varepsilon_{\theta}(x_t, t) \right)
$$

$$
\sigma_q(t) = \sqrt{\frac{(1 - \alpha_t)(1 - \bar{\alpha}_{t-1})}{1 - \bar{\alpha}_t}}
$$

これを時刻 $T$ からスタートして、1 つずつ前の時刻のデータを生成します。

また最後の時刻の処理$ x_1 $ から $ x_0 $ を生成する処理）は、ノイズの追加をなくすとよい結果が得られることがわかっています。そこで $ t = 1 $ のときは $ \varepsilon = 0 $ として、ノイズを追加しないようにします。以上のデータ生成のアルゴリズムは擬似コードで次のように表されます。

![](./picture/pict30.png)

